# Plagiarism Detection Model

Now that we've created training and test data, we are ready to define and train a model.

---

In [1]:
import os
import boto3
import pandas as pd
from sklearn.metrics import accuracy_score


import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker.pytorch import PyTorchModel

In [2]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

In [3]:
data_dir = 'plagiarism_data'
prefix = 'plagiarism_data'

data_path = sagemaker_session.upload_data(data_dir, bucket=bucket, key_prefix=prefix)

---
#### Below is the model that has been designed to identify plagiarism on the dataset created from the previous notebook.

---

In [32]:
!pygmentize source/train.py

import argparse
import json
import os
import pandas as pd

import torch
import torch.optim as optim
import torch.nn as nn
import torch.utils.data

from model import BinaryClassifier


def model_fn(model_dir):
    
    """Load the PyTorch model from the `model_dir` directory."""
    
    print("Loading model.")

    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BinaryClassifier(model_info['input_features'], model_info['hidden_dim'], model_info['output_dim'])

    model_path = os.path.join(model_dir, 'model.pth')
    with open(model_path, 'rb') as f:
        model.load_state_dict(torch.load(f))

    model.to(device).eval()
    
    print("Done loading model.")
    
    return model


def _get_train_data_loader(batch_size, training_dir):
    
   

In [42]:
estimator = PyTorch(
    entry_point='train.py',
    source_dir='source',
    role=role,
    framework_version='1.0',
    sagemaker_session=sagemaker_session,
    train_instance_count=1,
    train_instance_type='ml.c4.xlarge',
    hyperparameters={
        'input_features': 3,
        'hidden_dim': 32,
        'output_dim': 1,
        'epochs': 50
    }
)

## EXERCISE: Train the estimator

Train your estimator on the training data stored in S3. This should create a training job that you can monitor in your SageMaker console.

In [43]:
%%time
estimator.fit({'train': os.path.join(data_path, 'train.csv')})

2020-02-20 17:59:48 Starting - Starting the training job...
2020-02-20 17:59:50 Starting - Launching requested ML instances......
2020-02-20 18:00:58 Starting - Preparing the instances for training......
2020-02-20 18:02:16 Downloading - Downloading input data
2020-02-20 18:02:16 Training - Downloading the training image...
2020-02-20 18:02:46 Uploading - Uploading generated training modelbash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-02-20 18:02:37,260 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-02-20 18:02:37,262 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-02-20 18:02:37,274 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-02-20 18:02:40,290 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-02-20 18:02:40,700 sagemaker-containers INFO     Module train does 

In [48]:
%%time

model = PyTorchModel(
    model_data=estimator.model_data,
    role=role,
    framework_version='1.0',
    entry_point='predict.py',
    source_dir='source'
)

predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

---------------------!CPU times: user 467 ms, sys: 14.2 ms, total: 481 ms
Wall time: 10min 32s


In [56]:


accuracy = accuracy_score(test_y, test_y_preds)
print(accuracy)

## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

1.0

Predicted class labels: 
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


---
### Observations


#### There are no false positives or negatives currently. This could be due to the fact that the data sample is small; it could be, because there is a clear enough line between the answer and source texts that the model was able to correctly label across the board, or there could be some overfitting that has taken place. In reality it is likely a blend of these reasons. 

#### I chose to create a neural network utilizing PyTorch for a few reasons. I wanted to choose a good solution for the problem, not just a simple solution; and while linear regression models are good for binary classification problems and simpler to create and train, I've found that neural networks that are trained and developed properly can produce more robust results.

---

In [58]:
predictor.delete_endpoint()
bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()